# Lab: Aprendizaje por refuerzo multi-agente (III)

En el notebook 08 sobre JAL-GT se presenta al final el algoritmo JAL-AM que propone una solución al potencial problema de no disponer de acceso a la señal de recompensa de los otros agentes. En este caso, los métodos de modelado de agentes (_Agent Modelling, AM_) se pueden utilizar para inferir la política de los demás agentes via observación externa. JAL-AM garantiza convergencia a estrategias óptimas de _best response_ (mejor respuesta).

En determinados entornos, es posible que tampoco tengamos acceso a las observaciones sobre las acciones que los demás agentes están realizando sobre el entorno. En este notebook, vamos a explorar un método de gradiente de política que garantiza convergencia a equilibrio de Nash sin necesidad de tener acceso a señal de recompensa ni acciones realizadas de los demás agentes.

El método se llama WoLF-PHC: _Win or Learn Fast - Policy Hill Climbing_ y se puede consultar su diseño y su motivación a partir de la página 145 del libro [Multi-Agent Reinforcement Learning, Albrecht et al. 2024](https://www.marl-book.com/download/). La idea principal del algoritmo consiste en construir y actualizar tres elementos:

* El valor Q para el agente que entrena (como en Q-Learning, JAL-GT y JAL-AM).
* La política $\pi$ que se está aprendiendo.
* La política _media_ $\bar{\pi}$, que es una media de los valores más recientes de la política.

El motivo de tener, por separado, la política que se aprende y una media de las últimas políticas es que, para una parte substancial de juegos en forma normal y extensiva, esta política media converge en el infinito en un equilibrio de Nash del agente. Intuitivamente: esta política media contiene suficiente información temporal como para reflejar la dinámica de la convergencia hacia el equilibrio basándose en las recompensas de nuestro agente y de los cambios en las estrategias de los otros agentes. Más información sobre los fundamentos teóricos de estas propiedades se pueden encontrar en las Secciones 6.3.1 y 6.4.3 de [Multi-Agent Reinforcement Learning, Albrecht et al. 2024](https://www.marl-book.com/download/).

En cada iteración, actualizamos el valor Q (como en otros algoritmos ya vistos) y la política media. Después, se comprueba si el valor esperado actual (multiplicando política por valor Q) mejora o empeora a la política media. Se pueden dar dos casos:

* Mejoramos a la política media: en este caso, el gradiente de la política será pequeño (_loss learning rate_ o $lr_l$.
* No mejoramos a la política media: el gradiente será grande (_win learning rate_ o $lr_w$).

Esta es la parte a la que hace referencia el nombre del algoritmo: _win or learn fast_. Si estamos mejorando, modificamos la política muy poco y somos conservadores. Si no estamos mejorando, buscamos rápidamente un cambio a mejores utilidades esperadas, modificando sustancialmente la política. Este tipo de búsqueda se asemeja a _hill climbing_, y de ahí el sufijo PHC en el nombre.

El gradiente, una vez decidido si es el _loss_ o el _win learning rate_, se aplica sobre la acción con mejor valor Q, de manera positiva, y sobre las demás acciones, de manera negativa.

El pseudocódigo es el siguiente:

<div>
<img src="Algorithm9.png" width="500"/>
</div>

<div>
<img src="Algorithm9-eq.png" width="500"/>
</div>


## Instalación y configuración

### Dependencias

In [ ]:
!pip --quiet install rlcard pettingzoo seaborn matplotlib numpy pandas tinynn pygame

### Imports necesarios

In [ ]:
import abc
import itertools
import random

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pettingzoo.classic import rps_v2
from tinynn.core.layer import Dense, ReLU
from tinynn.core.loss import MSE
from tinynn.core.initializer import Zeros
from tinynn.core.model import Model
from tinynn.core.net import Net
from tinynn.core.optimizer import SGD

### Código de anteriores notebooks

In [ ]:
RPS_CHOICES = ["Rock", "Paper", "Scissors"]
PD_CHOICES = ["Cooperate", "Defect"]

def pretty_print_array(ar):
    return np.array_str(np.array(ar), precision=2, suppress_small=True)

def draw_history(history, title):
    data = pd.DataFrame({'Episode': range(1, len(history) + 1), title: history})
    plt.figure(figsize=(10, 6))
    sns.lineplot(x='Episode', y=title, data=data)

    plt.title(title + ' Over Episodes')
    plt.xlabel('Episode')
    plt.ylabel(title)
    plt.grid(True)
    plt.tight_layout()

    plt.show()

In [ ]:
class GameModel:
    def __init__(self, num_agents, num_states, num_actions):
        self.num_agents = num_agents
        self.num_states = num_states
        self.num_actions = num_actions
        self.action_space = self.generate_action_space()
        self.action_space_index = {joint_action: idx for idx, joint_action in enumerate(self.action_space)}

    def generate_action_space(self):
        actions_by_players = []
        for agent_id in range(self.num_agents):
            actions_by_players.append(range(self.num_actions))
        all_joint_actions = itertools.product(*actions_by_players)
        return [tuple(l) for l in all_joint_actions]

In [ ]:
class MARLAlgorithm(abc.ABC):
    @abc.abstractmethod
    def learn(self, joint_action, rewards, state, next_state):
        pass

    @abc.abstractmethod
    def select_action(self, state):
        pass

In [ ]:
def one_hot(state, max_states):
    one_hot_vector = np.zeros(max_states)
    one_hot_vector[state] = 1
    return np.array([one_hot_vector], dtype=float)

def normal_form_obs_to_state(observation):
    # Sólo un estado (juego en forma normal)
    return 0

def train(env, game, f_obs_to_state, algorithms):
    cumulative_rewards = [[0, 0]]
    actions_played = [[], []]
    all_agents = range(game.num_agents)

    observations, _ = env.reset()
    states = [f_obs_to_state(observations[f"player_{i}"]) for i in all_agents]

    while env.agents:
        joint_action = tuple([algorithms[i].select_action(states[i]) for i in all_agents])
        [actions_played[i].append(joint_action[i]) for i in all_agents]
        pettingzoo_joint_action = {f"player_{i}": joint_action[i] for i in all_agents}

        observations, rewards, terminations, truncations, infos = env.step(pettingzoo_joint_action)

        observations = [observations[f"player_{i}"] for i in all_agents]
        rewards = [rewards[f"player_{i}"] for i in all_agents]
        new_states = [f_obs_to_state(observations[i]) for i in all_agents]
        [algorithms[i].learn(joint_action, rewards, states[i], new_states[i])
         for i in all_agents]
        cumulative_rewards.append([cumulative_rewards[-1][i] + rewards[i] for i in all_agents])
        states = new_states

    return cumulative_rewards, actions_played

## Implementación del algoritmo WoLF-PHC

En primer lugar definimos una función de normalización para convertir cualquier lista de números reales en una distribución de probabilidad con suma total 1. Esta conversión es lineal, en comparación a softmax que hace una conversión exponencial.

In [ ]:
def normalize(x):
    min_val = np.min(x)
    if min_val < 0:
        x -= min_val
    total_sum = np.sum(x)
    if total_sum == 0:
        return [1/len(x)] * len(x)
    normalized_probabilities = np.array([p/total_sum for p in x])
    return normalized_probabilities

Ahora definimos la clase que implementa el algoritmo. Observad cómo usamos tres redes neuronales para poder capturar espacios de estados potencialmente complejos: una red para el valor Q, otra para la política, y otra para la política media:

In [ ]:
class WoLFPHC(MARLAlgorithm):
    def __init__(self, agent_id, game: GameModel, learning_rate_loss, learning_rate_win, gamma=0.95, alpha=0.5, epsilon=0.2, seed=42):
        assert(learning_rate_loss > learning_rate_win)
        self.agent_id = agent_id
        self.game = game
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.learning_rate_win = learning_rate_win
        self.learning_rate_loss = learning_rate_loss
        self.rng = random.Random(seed)
        net_q = Net([Dense(64, w_init=Zeros(), b_init=Zeros()),
                     ReLU(),
                     Dense(32, w_init=Zeros(), b_init=Zeros()),
                     ReLU(),
                     Dense(game.num_actions, w_init=Zeros(), b_init=Zeros())])
        self.q_model = Model(net=net_q, loss=MSE(), optimizer=SGD(lr=alpha))
        self.visit_counts = np.zeros(game.num_states)
        net_pi = Net([Dense(64, w_init=Zeros(), b_init=Zeros()),
                      ReLU(),
                      Dense(32, w_init=Zeros(), b_init=Zeros()),
                      ReLU(),
                      Dense(game.num_actions, w_init=Zeros(), b_init=Zeros())])
        self.policy = Model(net=net_pi, loss=MSE(), optimizer=SGD(lr=0.1))
        net_avg_pi = Net([Dense(64, w_init=Zeros(), b_init=Zeros()),
                          ReLU(),
                          Dense(32, w_init=Zeros(), b_init=Zeros()),
                          ReLU(),
                          Dense(game.num_actions, w_init=Zeros(), b_init=Zeros())])
        self.average_policy = Model(net=net_avg_pi, loss=MSE(), optimizer=SGD(lr=0.1))
        self.metrics = {'td_error': [], 'loss': []}

El `learning rate` de las redes de política y política media debería ser diferente de la tasa de aprendizaje para el valor Q, debido a que los gradientes de política en este algoritmo son fijos. Por lo tanto, podemos dejar estas dos tasas prefijadas y controlar la convergencia del algoritmo por las dos tasas de aprendizaje $lr_l$ y $lr_w$.

A continuación añadimos el método para actualizar el valor Q, que es idéntico al visto en el notebook anterior para JAL-GT:

In [ ]:
class WoLFPHC(WoLFPHC):
    def update_q(self, q_model, state, action, reward, next_value):
        prediction = q_model.forward(state)[0]
        agent_q_value = prediction[action]
        td_target = reward + self.gamma * next_value
        td_error = td_target - agent_q_value
        target = prediction.copy()
        target[action] = td_target
        loss, grads = q_model.backward(np.array([prediction]), np.array([target]))
        q_model.apply_grads(grads)
        return td_error, loss

El siguiente paso según el pseudocódigo es actualizar la política media, que en el infinito debería converger en una estrategia de equilibrio de Nash. Observad que al estar tratando con distribuciones de probabilidad (lo que es una política) necesitamos normalizar en determinados puntos clave:

In [ ]:
class WoLFPHC(WoLFPHC):
    def update_average_policy(self, scalar_state):
        state = one_hot(scalar_state, self.game.num_states)
        avg_pi_s = self.average_policy.forward(state)[0]
        current_pi_s = normalize(self.policy.forward(state)[0])
        target_avg_pi_s = avg_pi_s.copy()
        for a_i in range(self.game.num_actions):
            pi_a_i = current_pi_s[a_i]
            # Sumamos el gradiente del error: diferencia entre política actual y política media,
            # ponderado por el número de visitas en el estado para converger en un valor estable
            target_avg_pi_s[a_i] += (pi_a_i - normalize(avg_pi_s)[a_i]) / self.visit_counts[scalar_state]
        target_avg_pi_s = normalize(target_avg_pi_s)
        # Actualizamos la red neuronal
        _, grads = self.average_policy.backward(np.array([avg_pi_s]), np.array([target_avg_pi_s]))
        self.average_policy.apply_grads(grads)

Después de actualizar la política media toca actualizar la política que se está aprendiendo. En este caso juntamos las ecuaciones 6.44 a 6.46:

* Calculamos la utilidad esperada en el estado, para la política y para la política media (multiplicación de Q por cada distribución de probabilidad).
* Dependiendo de si estamos ganando o no a la política media, decidimos una tasa de aprendizaje que será nuestro gradiente.
* Aplicamos el gradiente a cada acción, positivamente a las mejores acciones, negativamente a las demás.
* Actualizamos la red neuronal en base a estos gradientes.

In [ ]:
class WoLFPHC(WoLFPHC):
    def update_policy(self, scalar_state):
        state = one_hot(scalar_state, self.game.num_states)

        current_policy_s = self.policy.forward(state)[0]
        current_avg_pi_s = self.average_policy.forward(state)[0]
        q_values_s = self.q_model.forward(state)[0]

        # Calculamos delta (ecuación 6.46)
        policy_value = np.sum(np.multiply(normalize(current_policy_s), q_values_s))
        avg_pi_value = np.sum(np.multiply(normalize(current_avg_pi_s), q_values_s))
        if policy_value > avg_pi_value:
            delta = self.learning_rate_win
        else:
            delta = self.learning_rate_loss

        # Calculamos delta para (s, a_i) (ecuación 6.45)
        delta_all = [min(normalize(current_policy_s)[a_i], delta / (self.game.num_actions - 1)) for a_i in range(self.game.num_actions)]

        # Comprobamos qué acción es la mejor según Q y con qué valor
        q_values_state = self.q_model.forward(state)[0]
        max_q = np.max(q_values_state)
        target_policy_s = current_policy_s.copy()

        # Recorremos las acciones, aplicando delta
        for a_i in range(self.game.num_actions):
            if q_values_state[a_i] == max_q:
                # Mejores acciones, aplicamos positivamente
                target_policy_s[a_i] += sum([delta_all[a_ii] if a_ii != a_i else 0 for a_ii in range(self.game.num_actions)])
            else:
                # Demás acciones, aplicamos negativamente
                target_policy_s[a_i] -= delta_all[a_i]
        # Actualizamos la red neuronal
        target_policy_s = normalize(target_policy_s)
        loss, grads = self.policy.backward(np.array([current_policy_s]), np.array([target_policy_s]))
        self.policy.apply_grads(grads)

Finalmente, implementamos los métodos públicos: `learn` y `select_action`.

In [ ]:
class WoLFPHC(WoLFPHC):
    def learn(self, joint_action, rewards, state, next_state):
        # Sólo consideramos lo que respecta a nuestro agente
        agent_action = joint_action[self.agent_id]
        agent_reward = rewards[self.agent_id]
        one_hot_state = one_hot(state, self.game.num_states)
        one_hot_next_state = one_hot(next_state, self.game.num_states)
        next_value = np.max(self.q_model.forward(one_hot_next_state)[0])

        # Actualizamos valor Q
        td_error, loss = self.update_q(self.q_model, one_hot_state, agent_action, agent_reward, next_value)

        # Actualizamos política media
        self.visit_counts[state] += 1
        self.update_average_policy(state)
        self.update_policy(state)

        # Guardamos el error de diferencia temporal y la pérdida de la red neuronal para estadísticas posteriores
        self.metrics['td_error'].append(abs(td_error))
        self.metrics['loss'].append(abs(loss))

    def select_action(self, state, train=True):
        if train and self.rng.random() < self.epsilon:
                return self.rng.choice(range(self.game.num_actions))
        else:
            np.random.seed(self.rng.randint(0, 10000))
            return np.random.choice(range(self.game.num_actions), p=normalize(self.q_model.forward(one_hot(state, self.game.num_states))[0]))

## Experimento: Rock, Paper, Scissors

In [ ]:
def train_rps_2_agents(num_turns, learning_rates_loss, learning_rates_win,
                       gammas, alphas, epsilons, seeds):
    game_model = GameModel(num_agents=2, num_states=1, num_actions=3)
    algorithm_player_0 = WoLFPHC(0, game_model,
                                 learning_rate_loss=learning_rates_loss[0],
                                 learning_rate_win=learning_rates_win[0],
                                 gamma=gammas[0], alpha=alphas[0], epsilon=epsilons[0], seed=seeds[0])
    algorithm_player_1 = WoLFPHC(1, game_model,
                                 learning_rate_loss=learning_rates_loss[1],
                                 learning_rate_win=learning_rates_win[1],
                                 gamma=gammas[1], alpha=alphas[1], epsilon=epsilons[1], seed=seeds[1])
    env = rps_v2.parallel_env(max_cycles=num_turns, render_mode="ansi")

    cumulative_rewards, actions_played = train(env, game_model, normal_form_obs_to_state,
                                               [algorithm_player_0, algorithm_player_1])

    env.close()
    return game_model, algorithm_player_0, algorithm_player_1, cumulative_rewards, actions_played

In [ ]:
game_model, algorithm_player_0, algorithm_player_1, cumulative_rewards, actions_played = \
        train_rps_2_agents(num_turns=15, learning_rates_loss=[0.001, 0.001],
                           learning_rates_win=[0.00001, 0.00001],
                           gammas=[0.95, 0.95], alphas=[0.0001, 0.0001],
                           epsilons=[0.2, 0.2], seeds=[0, 1])

# Recompensa acumulada. Debería ser [0, 0] en el infinito, si las estrategias son óptimas
print(f"Recompensas acumuladas: {cumulative_rewards[-1][0]}, {cumulative_rewards[-1][1]}")

# Espacio de acciones conjuntas
print(f"Espacio de acciones conjuntas ordenado: {game_model.action_space}")

In [ ]:
# Valores Q calculados por los dos agentes:
print("Valores Q calculados por el agente 0:")
print(pretty_print_array(algorithm_player_0.q_model.forward(one_hot(0, game_model.num_states))[0]))
print("Valores Q calculados por el agente 1:")
print(pretty_print_array(algorithm_player_1.q_model.forward(one_hot(0, game_model.num_states))[0]))

# Política del agente 0:
print(f"Política del agente 0: {normalize(algorithm_player_0.policy.forward(one_hot(0, game_model.num_states))[0])}")

# Política del agente 1:
print(f"Política del agente 1: {normalize(algorithm_player_1.policy.forward(one_hot(0, game_model.num_states))[0])}")

In [ ]:
draw_history(algorithm_player_0.metrics["td_error"], "TD Error")

In [ ]:
draw_history(algorithm_player_0.metrics["loss"], "Loss")

## Experimento: dilema del prisionero

In [ ]:
import prisoners_dilemma

In [ ]:
def train_pd(num_turns, learning_rates_loss, learning_rates_win, gammas, alphas, epsilons, seeds):
    game_model = GameModel(num_agents=2, num_states=1, num_actions=2)
    algorithm_player_0 = WoLFPHC(0, game_model,
                                 learning_rate_loss=learning_rates_loss[0],
                                 learning_rate_win=learning_rates_win[0],
                                 gamma=gammas[0], alpha=alphas[0], epsilon=epsilons[0], seed=seeds[0])
    algorithm_player_1 = WoLFPHC(1, game_model,
                                 learning_rate_loss=learning_rates_loss[1],
                                 learning_rate_win=learning_rates_win[1],
                                 gamma=gammas[1], alpha=alphas[1], epsilon=epsilons[1], seed=seeds[1])
    env = prisoners_dilemma.parallel_env(max_cycles=num_turns, render_mode="ansi")
    cumulative_rewards, actions_played = train(env, game_model, normal_form_obs_to_state,
                                               [algorithm_player_0, algorithm_player_1])
    return game_model, algorithm_player_0, algorithm_player_1, cumulative_rewards, actions_played

In [ ]:
game_model, algorithm_player_0, algorithm_player_1, cumulative_rewards, actions_played = \
    train_pd(num_turns=1, learning_rates_loss=[0.001, 0.001], learning_rates_win=[0.00001, 0.00001],
             gammas=[0.95, 0.95], alphas=[0.01, 0.01], epsilons=[0.2, 0.2], seeds=[0, 1])

In [ ]:
# Recompensa acumulada
print(f"Recompensas acumuladas: {cumulative_rewards[-1][0]}, {cumulative_rewards[-1][1]}")

# Espacio de acciones conjuntas
print(f"Espacio de acciones conjuntas ordenado: {game_model.action_space}")

# Valores Q calculados por los dos agentes:
print("Valores Q calculados por el agente 0:")
print(pretty_print_array(algorithm_player_0.q_model.forward(one_hot(0, game_model.num_states))))
print("Valores Q calculados por el agente 1:")
print(pretty_print_array(algorithm_player_1.q_model.forward(one_hot(0, game_model.num_states))))

# Política del agente 0:
print(f"Política del agente 0: {normalize(algorithm_player_0.policy.forward(one_hot(0, game_model.num_states))[0])}")

# Política del agente 1:
print(f"Política del agente 1: {normalize(algorithm_player_1.policy.forward(one_hot(0, game_model.num_states))[0])}")

In [ ]:
draw_history(algorithm_player_0.metrics["td_error"], "TD Error")

In [ ]:
draw_history(algorithm_player_0.metrics["loss"], "Loss")